In [ ]:
import os
import torch
import torchvision.transforms as transforms
from google.colab import drive
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drive.mount('/content/drive')
validation_dataset_path = "/content/drive/MyDrive/test_images" #test dataset for analysis
model_path = "/content/drive/MyDrive/efficient_net_models/model_5epochs"

Mounted at /content/drive


In [ ]:
model = torch.load(model_path)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
#transformation class to create object of
class ImageTransformer:
    def __init__(self):
        self._image_size = (224, 224)

        self.transform = transforms.Compose([
            transforms.Resize(self._image_size),
            transforms.ToTensor(),
            # transforms.Normalize(mean=self._mean, std=self._std)
        ])

In [ ]:
#transforms image into what required as input for model
image_processor = ImageTransformer()


In [ ]:
id_to_label = {0: 'Packing_List', 1: 'Bill_of_lading', 2: 'Certificate_Of_Origin',
                  3: 'Insurance_Certificate', 4: 'Covering_Schedule'}

In [ ]:
import pandas as pd
a = pd.DataFrame
data ={"File_Name": [] ,"Actual_Class": [], "Predicted_Class": []}
from PIL import Image
for file in os.listdir(validation_dataset_path):
  img = Image.open(os.path.join(validation_dataset_path,file)).convert("RGB")
  #print(file[:-13])
  transformed_input_image = image_processor.transform(img)
  transformed_input_image = transformed_input_image.reshape(1,3,224,224)
  #print(transformed_input_image.shape)
  output = model(transformed_input_image)
  _, predicted_idx = torch.max(output, 1)
  #print(output.shape)
  data["File_Name"].append(file)
  actual_class = file[:-13]
  if actual_class[-1] == "_":
    actual_class = actual_class[:-1]
  data["Actual_Class"].append(actual_class)
  data["Predicted_Class"].append(id_to_label[int(predicted_idx)])
  #print(id_to_label[int(predicted_idx)])
a = pd.DataFrame(data)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
classes = a['Actual_Class'].unique()
precisions = []
recalls = []
f1_scores = []

for cls in classes:
    actual = a['Actual_Class'].map(lambda x: 1 if x == cls else 0)
    predicted = a['Predicted_Class'].map(lambda x: 1 if x == cls else 0)

    precision = precision_score(actual, predicted)
    recall = recall_score(actual, predicted)
    f1 = f1_score(actual, predicted)

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

In [ ]:
metrics_df = pd.DataFrame({
    'Class': classes,
    'Precision': precisions,
    'Recall': recalls,
    'F1-Score': f1_scores
})

print(metrics_df)

                   Class  Precision  Recall  F1-Score
0           Packing_List   0.000000    0.00  0.000000
1         Bill_of_lading   0.000000    0.00  0.000000
2  Certificate_Of_Origin   0.000000    0.00  0.000000
3  Insurance_Certificate   0.230769    0.75  0.352941
4      Covering_Schedule   0.000000    0.00  0.000000


In [ ]:
metrics_df.to_csv("metric_df.csv")

In [ ]:
a.to_csv("Results.csv")